In [ ]:
# !pip install arch

In [ ]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt

In [ ]:
from google.colab import files

uploaded = files.upload()

for returns in uploaded.keys():
  with open(returns, 'rb') as f:
    pass


Saving 지윤 누님 엑셀 수익률 자료.xlsx to 지윤 누님 엑셀 수익률 자료 (1).xlsx


In [ ]:
# @title
# 개별종목별 수익률
returns_each = pd.read_excel(returns,index_col=0)

# 포트폴리오 투자금액 : 100억
values = 10000000000

# 종목별 비중
proportion = np.array([0.4736, 0.159, 0.0896, 0.0592, 0.056, 0.0512, 0.0433, 0.0343, 0.0338])

# 종목별 MV
mv = [values*proportion_i for proportion_i in proportion]

# 포트폴리오 손익(loss and gain) = 종목별 MV * 종목별 수익률
lossngain = [(mv * returns_each.values).sum(axis=1)]
#print(np.sort(lossngain))

# HSA으로 신뢰구간 95% 및 99% VaR 계산
hsa_Var_95 = np.percentile(lossngain, 5, method = 'lower')
hsa_Var_99 = np.percentile(lossngain, 1, method = 'lower')

print('The historical simulation approach 95% VaR = ' + str(hsa_Var_95))
print('The historical simulation approach 99% VaR = ' + str(hsa_Var_99))

The historical simulation approach 95% VaR = -199473510.27495432
The historical simulation approach 99% VaR = -268466136.05096596


In [ ]:
# 최근의 관측치에 가중치 부여(lamda = 0.9950),
lamda = 0.940
lossngain = np.array([(mv * returns_each.values).sum(axis=1)]).reshape(500,1)
#print(type(lossngain))
n = len(lossngain)
weights = np.array([(lamda**(n-i) * (1-lamda))/(1-lamda**n) for i in range(1, n+1)]).reshape(500,1)

# 가중치를 부여한 포트폴리오의 손익 lossngain_weighted
lossngain_weighted = np.concatenate([lossngain, weights], axis=1)

col_names = ['lossngain', 'weights']

lossngain_weighted = pd.DataFrame(lossngain_weighted, columns=col_names)

lossngain_weighted_sort = lossngain_weighted.sort_values(by='lossngain', ascending=True)

lossngain_weighted_sort['Accumulatedweights'] = lossngain_weighted_sort['weights'].cumsum()

#print(lossngain_weighted_sort.head(20))

# 가중치를 포함해서 신뢰구간 95% 및 99% VaR 계산
weighted_Var_95 = np.array(lossngain_weighted_sort[lossngain_weighted_sort['Accumulatedweights'] > 0.05].head(1)['lossngain'])
weighted_Var_99 = np.array(lossngain_weighted_sort[lossngain_weighted_sort['Accumulatedweights'] > 0.01].head(1)['lossngain'])

print('The different weighting 95% VaR = ' + str(weighted_Var_95[0]))
print('The different weighting 99% VaR = ' + str(weighted_Var_99[0]))

The different weighting 95% VaR = -225804928.14814186
The different weighting 99% VaR = -266355769.14033362


In [ ]:
from google.colab import files

uploaded = files.upload()

for closing in uploaded.keys():
  with open(closing, 'rb') as f:
    pass

# 개별종목별 수정종가
closing_prices = pd.read_excel(closing,index_col=0)

Saving 지윤 누님 엑셀 자료.xlsx to 지윤 누님 엑셀 자료 (2).xlsx


In [ ]:
import arch
from arch import arch_model
volatility = pd.DataFrame(index=returns_each.index)

for col in returns_each.columns:
  returns_data = returns_each[col]
  model = arch.arch_model(returns_data, vol='GARCH', p=1, q=1, rescale=False)
  results = model.fit()
  forecasts = results.forecast(start=0, reindex=False)
  volatility[col] = np.array([(np.sqrt(forecasts.variance))]).reshape(500,)

vol_tomorrow = volatility.iloc[-1].copy().values

#print(vol_tomorrow)

returns_vol = pd.DataFrame(index=returns_each.index)

for i, col_name in enumerate(closing_prices.columns):
  temp_returns = []
  for j in range(500):
    #print(closing_prices[col_name][j])
    a = closing_prices[col_name][j]
    b = closing_prices[col_name][j+1]
    c = volatility[col_name][j]
    #print((a+(b-a)*vol_tomorrow[i]/c)/a - 1)
    temp_returns.append((a+(b-a)*vol_tomorrow[i]/c)/a - 1)
  returns_vol[col_name] = np.array(temp_returns)

#print(returns_vol)
# 포트폴리오 손익(loss and gain) = 종목별 MV * 종목별 수익률
lossngain_vol = [(mv * returns_vol.values).sum(axis=1)]
#print(lossngain_vol)
#print(np.sort(lossngain_vol))

Iteration:      1,   Func. Count:      6,   Neg. LLF: 237203260672.85492
Iteration:      2,   Func. Count:     18,   Neg. LLF: 126346100.8180959
Optimization terminated successfully    (Exit mode 0)
            Current function value: -1394.3495418197995
            Iterations: 2
            Function evaluations: 27
            Gradient evaluations: 2
Iteration:      1,   Func. Count:      6,   Neg. LLF: 25845.073872384957
Iteration:      2,   Func. Count:     16,   Neg. LLF: 1721.766255174798
Iteration:      3,   Func. Count:     25,   Neg. LLF: 2110.2143650234716
Iteration:      4,   Func. Count:     35,   Neg. LLF: 367523956.7655958
Iteration:      5,   Func. Count:     43,   Neg. LLF: -1172.7207443305329
Iteration:      6,   Func. Count:     49,   Neg. LLF: -1173.808968092335
Iteration:      7,   Func. Count:     55,   Neg. LLF: -1176.7048148315555
Iteration:      8,   Func. Count:     61,   Neg. LLF: 26073.265685698992
Iteration:      9,   Func. Count:     72,   Neg. LLF: 669.2568

In [ ]:
# 변동성 updating으로 신뢰구간 95% 및 99% VaR 계산
vol_Var_95 = np.percentile(lossngain_vol, 5, method = 'lower')
vol_Var_99 = np.percentile(lossngain_vol, 1, method = 'lower')

print('The volatility updating 95% VaR = ' + str(vol_Var_95))
print('The volatility updating 99% VaR = ' + str(vol_Var_99))

The volatility updating 95% VaR = -206463435.3838949
The volatility updating 99% VaR = -278390335.0876013


In [ ]:
# 변동성과 가중치를 포함한 포트폴리오의 손익 lossngain_weighted
lossngain_vol_weighted = np.concatenate([np.array(lossngain_vol).T, weights], axis=1)

col_names = ['lossngain_vol', 'weights']

lossngain_vol_weighted = pd.DataFrame(lossngain_vol_weighted, columns=col_names)

#손실액 순으로 sort
lossngain_vol_weighted_sort = lossngain_vol_weighted.sort_values(by='lossngain_vol', ascending=True)

#누적 가중치 계산
lossngain_vol_weighted_sort['Accumulatedweights'] = lossngain_vol_weighted_sort['weights'].cumsum()

#print(lossngain_vol_weighted_sort.head(20))

# 변동성과 가중치를 포함해서 신뢰구간 95% 및 99% VaR 계산
vol_weighted_Var_95 = np.array(lossngain_vol_weighted_sort[lossngain_vol_weighted_sort['Accumulatedweights'] > 0.05].head(1)['lossngain_vol'])
vol_weighted_Var_99 = np.array(lossngain_vol_weighted_sort[lossngain_vol_weighted_sort['Accumulatedweights'] > 0.01].head(1)['lossngain_vol'])

print('The different weighting & volatility updating 95% VaR = ' + str(vol_weighted_Var_95[0]))
print('The different weighting & volatility updating 99% VaR = ' + str(vol_weighted_Var_99[0]))

The different weighting & volatility updating 95% VaR = -225944936.16043028
The different weighting & volatility updating 99% VaR = -266022404.97770435


In [ ]:
# 부트스트랩 검증 데이터 생성
def sample(data):
  # Flatten the data to 1-dimension if it's not
  if isinstance(data, pd.DataFrame): # Check if data is DataFrame
    data = data.values.flatten() # Extract values as NumPy array and flatten
  elif data.ndim > 1:
    data = data.flatten()

  sample = np.random.choice(data, len(data), replace=True)
  sample_VaR_95 = np.percentile(sample, 5, method ='higher')
  sample_VaR_99 = np.percentile(sample, 1, method ='higher')

  sample_weighted = np.concatenate([np.array(sample).reshape(-1, 1), weights], axis=1)
  col_names = ['lossngain', 'weights']

  sample_weighted = pd.DataFrame(sample_weighted, columns=col_names) # Convert to DataFrame for sorting

  sample_weighted_sort = sample_weighted.sort_values(by='lossngain', ascending=True)
  sample_weighted_sort['Accumulatedweights'] = sample_weighted_sort['weights'].cumsum()

  weighted_Var_95 = np.array(sample_weighted_sort[sample_weighted_sort['Accumulatedweights'] > 0.05].head(1)['lossngain'])
  weighted_Var_99 = np.array(sample_weighted_sort[sample_weighted_sort['Accumulatedweights'] > 0.01].head(1)['lossngain'])

  sample_weighted_VaR_95 = weighted_Var_95[0]
  sample_weighted_VaR_99 = weighted_Var_99[0]

  return sample_VaR_95, sample_VaR_99, sample_weighted_VaR_95, sample_weighted_VaR_99

# HSA 부트스트랩 검증 함수
def bootstrap_VaR(func, data):
  bootstrap_VaR_95 = []
  bootstrap_VaR_99 = []
  bootstrap_weighted_VaR_95 = []
  bootstrap_weighted_VaR_99 = []

  for i in range(1000):
    # Unpack the tuple returned by func(data)
    s_VaR_95, s_VaR_99, s_w_VaR_95, s_w_VaR_99 = func(data)
    bootstrap_VaR_95.append(s_VaR_95)
    bootstrap_VaR_99.append(s_VaR_99)
    bootstrap_weighted_VaR_95.append(s_w_VaR_95)
    bootstrap_weighted_VaR_99.append(s_w_VaR_99)

  #print(bootstrap_VaR_95)
  #print(bootstrap_weighted_VaR_95)
  # 베이직한 값과 가중치를 부여한 값의 신뢰구간이 같아서 raw데이터를 봤는데 이상없음

  # Convert the list elements to numeric type before calculating mean and std
  bootstrap_VaR_95 = np.array(bootstrap_VaR_95, dtype=np.float64)
  bootstrap_VaR_99 = np.array(bootstrap_VaR_99, dtype=np.float64)
  bootstrap_weighted_VaR_95 = np.array(bootstrap_weighted_VaR_95, dtype=np.float64)
  bootstrap_weighted_VaR_99 = np.array(bootstrap_weighted_VaR_99, dtype=np.float64)

  min_95 = np.mean(bootstrap_VaR_95)-2*np.std(bootstrap_VaR_95)
  max_95 = np.mean(bootstrap_VaR_95)+2*np.std(bootstrap_VaR_95)

  min_99 = np.mean(bootstrap_VaR_99)-2*np.std(bootstrap_VaR_99)
  max_99 = np.mean(bootstrap_VaR_99)+2*np.std(bootstrap_VaR_99)

  weighted_min_95 = np.mean(bootstrap_weighted_VaR_95)-2*np.std(bootstrap_weighted_VaR_95)
  weighted_max_95 = np.mean(bootstrap_weighted_VaR_95)+2*np.std(bootstrap_weighted_VaR_95)

  weighted_min_99 = np.mean(bootstrap_weighted_VaR_99)-2*np.std(bootstrap_weighted_VaR_99)
  weighted_max_99 = np.mean(bootstrap_weighted_VaR_99)+2*np.std(bootstrap_weighted_VaR_99)

  print(f'The Bootstrap 95% confidence interval {min_95},{max_95})' )
  print(f'The Bootstrap 99% confidence interval {min_99},{max_99})' )
  print(f'The Bootstrap different weighting 95% confidence interval {min_95},{max_95})' )
  print(f'The Bootstrap different weighting 99% confidence interval {min_99},{max_99})' )



print('The historical simulation approach')
bootstrap_VaR(sample, lossngain)
print('\n')
print('The historical simulation approach volatility updating')
bootstrap_VaR(sample, lossngain)


The historical simulation approach
The Bootstrap 95% confidence interval -220303969.58658177,-170484635.79108387)
The Bootstrap 99% confidence interval -294921549.82006466,-238212663.49796265)
The Bootstrap different weighting 95% confidence interval -220303969.58658177,-170484635.79108387)
The Bootstrap different weighting 99% confidence interval -294921549.82006466,-238212663.49796265)


The historical simulation approach volatility updating
The Bootstrap 95% confidence interval -221592925.59080878,-170599089.3789914)
The Bootstrap 99% confidence interval -294859928.8749384,-238761740.05975744)
The Bootstrap different weighting 95% confidence interval -221592925.59080878,-170599089.3789914)
The Bootstrap different weighting 99% confidence interval -294859928.8749384,-238761740.05975744)
